<a href="https://colab.research.google.com/github/davidbooker8/Computer-Graphics-David-Booker/blob/master/COSC6315_Mod_5_Big_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [ ]:
# Simple Linear Regression
from google.colab import files
uploaded = files.upload()

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
dataset = spark.read.csv('BostonHousing (1).csv', inferSchema= True, header= True)

In [ ]:
dataset.printSchema()

In [ ]:
assembler = VectorAssembler(inputCols= ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol= 'Attributes')
output = assembler.transform(dataset)

finalized_data = output.select('Attributes', 'medv')
finalized_data.show()

+--------------------+----+
|          Attributes|medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03237,0.0,2.18...|33.4|
|[0.06905,0.0,2.18...|36.2|
|[0.02985,0.0,2.18...|28.7|
|[0.08829,12.5,7.8...|22.9|
|[0.14455,12.5,7.8...|27.1|
|[0.21124,12.5,7.8...|16.5|
|[0.17004,12.5,7.8...|18.9|
|[0.22489,12.5,7.8...|15.0|
|[0.11747,12.5,7.8...|18.9|
|[0.09378,12.5,7.8...|21.7|
|[0.62976,0.0,8.14...|20.4|
|[0.63796,0.0,8.14...|18.2|
|[0.62739,0.0,8.14...|19.9|
|[1.05393,0.0,8.14...|23.1|
|[0.7842,0.0,8.14,...|17.5|
|[0.80271,0.0,8.14...|20.2|
|[0.7258,0.0,8.14,...|18.2|
+--------------------+----+
only showing top 20 rows



In [ ]:
from inspect import trace
#Split training data and targeting data
train_data, test_data = finalized_data.randomSplit([0.7, 0.3])

regressor = LinearRegression(featuresCol= 'Attributes', labelCol= 'medv')

#Learn to fit the model from training set
regressor = regressor.fit(train_data)
pred = regressor.evaluate(test_data)
pred.predictions.show()

+--------------------+----+------------------+
|          Attributes|medv|        prediction|
+--------------------+----+------------------+
|[0.00632,18.0,2.3...|24.0|30.425180697228036|
|[0.01301,35.0,1.5...|32.7|29.133211203433238|
|[0.01381,80.0,0.4...|50.0|  39.8783373323502|
|[0.01439,60.0,2.9...|29.1|31.428756488653228|
|[0.01501,90.0,1.2...|50.0| 42.37725537144886|
|[0.01538,90.0,3.7...|44.0| 36.99784959940642|
|[0.01709,90.0,2.0...|30.1| 23.71996168374204|
|[0.01778,95.0,1.4...|32.9|29.758175350858878|
|[0.0187,85.0,4.15...|23.1| 24.63245265421099|
|[0.01951,17.5,1.3...|33.0|21.472814115619123|
|[0.02055,85.0,0.7...|24.7| 24.32314806524102|
|[0.02177,82.5,2.0...|42.3| 36.02586379887973|
|[0.02731,0.0,7.07...|21.6|  25.0633687382684|
|[0.02899,40.0,1.2...|26.6|20.758150676707672|
|[0.02985,0.0,2.18...|28.7|25.130328997580158|
|[0.03041,0.0,5.19...|18.5|19.396988117005314|
|[0.03049,55.0,3.7...|31.2|27.970343288421837|
|[0.03237,0.0,2.18...|33.4|27.930957136991907|
|[0.03445,82.

In [ ]:
coeff= regressor.coefficients
intr = regressor.intercept
print("The coefficients of the model is: %a" %coeff)
print(" The intercept of the model is: %f" %intr)

The coefficients of the model is: DenseVector([-0.0992, 0.0466, 0.0631, 4.0356, -13.7561, 4.7189, -0.0038, -1.3007, 0.2547, -0.0115, -0.8174, 0.0121, -0.4986])
 The intercept of the model is: 23.972061


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol= "medv", predictionCol= "prediction", metricName= "rmse")
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)
#Root MSE, MAE, COD
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("R2: %.3f" % r2)


RMSE: 4.699
MSE: 22.084
MAE: 3.156
R2: 0.680
